In [16]:
import secretflow as sf

# In case you have a running secretflow runtime already.
sf.shutdown()
# 基于这四个party，我们将会建立3个设备。
# 一个基于 alice 的PYU设备
# 一个基于 dave 的PYU设备
# 一个基于 alice , bob 和 carol 的SPU设备
sf.init(['alice', 'bob', 'carol', 'dave'], address='local')

2023-06-01 20:30:04,996	INFO worker.py:1538 -- Started a local Ray instance.


In [17]:
# sf.utils.testing.cluster_def 是一个helper通过寻找未占用的端口来创建一个设置。
aby3_config = sf.utils.testing.cluster_def(parties=['alice', 'bob', 'carol'])

aby3_config

{'nodes': [{'party': 'alice', 'address': '127.0.0.1:43505'},
  {'party': 'bob', 'address': '127.0.0.1:46135'},
  {'party': 'carol', 'address': '127.0.0.1:48535'}],
 'runtime_config': {'protocol': 3, 'field': 3}}

In [18]:
# 随后我们用 aby3_config 来创建一个SPU设备并检查其 cluster_def 。
spu_device = sf.SPU(aby3_config)

spu_device.cluster_def

{'nodes': [{'party': 'alice', 'address': '127.0.0.1:43505'},
  {'party': 'bob', 'address': '127.0.0.1:46135'},
  {'party': 'carol', 'address': '127.0.0.1:48535'}],
 'runtime_config': {'protocol': 3, 'field': 3}}

In [19]:
# 最后，我们创建两个PYU设备。
alice, dave = sf.PYU('alice'), sf.PYU('dave')

In [20]:
# 这个block让我们理解如何将一个 Python object / PYUObject 传给一个SPU设备。
# 我试了直接把这玩意当字典，或者直接当数组都不能输出官方文档里的，我服了
# bank_account = {'id': 12345, 'deposit': 1000.25}, {'id': 12345, 'deposit': 100000.25}
bank_account = [{'id': 12345, 'deposit': 1000.25}, {'id': 12345, 'deposit': 100000.25}]

# 这个东西他先to给了alice然后再to给了spu单元，就是让alice做pyu的意思？ 不是 傻逼了 前面定义了
temp = sf.to(alice,bank_account)
bank_account_spu = sf.to(alice, bank_account).to(spu_device)
# temp就是个pyu的设备
print(temp)
type(bank_account_spu)

secretflow.device.device.spu.SPUObject

(_run pid=26732) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=26732) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=26732) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
(_run pid=26732) INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
(_run pid=26732) WARNING:jax._src.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [13]:
# 每一个SPUObject包含两个成员：
# meta
# shares
# 此时，由于我们是在 Host 创建一个SPU object。我们可以自由地查看这两个成员。
# 我们首先查看meta。
# meta保留了原始数据的结构，只是将数字和数列替换为 SPUValueMeta 。
# 这里好像也和官网不太一样
bank_account_spu.meta

ObjectRef(e0dc174c83599034ffffffffffffffffffffffff0100000001000000)

In [12]:
# 随后我们检查bank_account_spu的*shares* 。 由于我们将数据传递至一个3PC的SPU设备。我们可以有三个分片，我们将会检查第一个分片。
assert len(bank_account_spu.shares_name) == 3

# dir(bank_account_spu)
# 我不知道这里为什么和官网上输出的不一样
dir(bank_account_spu.shares_name[0])
bank_account_spu

In [11]:
bank_account_spu.shares_name[0]

ObjectRef(f4402ec78d3a260768c28136959f2a0cc8f182be0100000001000000)

In [9]:
# import pprint
# pprint.pprint(dir(bank_account_spu))
print(bank_account_spu.to)

<bound method DeviceObject.to of <secretflow.device.device.spu.SPUObject object at 0x7fdb082f7760>>


In [10]:
import pprint
pprint.pprint(dir(bank_account_spu))

['__abstractmethods__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'device',
 'device_type',
 'meta',
 'shares_name',
 'to']


In [14]:
# 关掉这玩意
sf.shutdown()